# Import libraries

In [ ]:
import numpy as np
from tensorflow.keras.applications import EfficientNetB1
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
print(tf.__version__)

In [ ]:
img_size = 240
num_classes = 16

In [ ]:
model = EfficientNetB1(
        include_top=False,
        classes=num_classes,
        input_shape=(img_size, img_size, 3),
        weights='imagenet',
        drop_connect_rate=0.4
)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

model.summary()

# Import tensorflow fashion_mnist dataset

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Begin with preprocessing of data

In [ ]:
plt.figure()
plt.imshow(train_images[1])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input, Dropout
from tensorflow.keras.regularizers import L1, L2

In [ ]:
model = tf.keras.Sequential([
    Flatten(input_shape=(28,28)),
    Dense(512, 
          activation='relu'),
    Dense(256,
          activation='relu'),
    Dropout(0.5),
    Dense(128,
          activation='relu'),
    Dense(10)
])

Here, the Sequentual model is making use of 4 layers:
- Input: the top layer in which it would like to take in the dimensions of said images
- Flatten: the layer in which grabs the 28x28 pixel images, and flattens them into one 2D tensor
- Dense(128), a 128 neuron layer that will process the pixels of these images
- Dense(10) a 10 neuron layer that will determine the classification of the clothing item

## Compile the model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(train_images, 
          train_labels, 
          epochs=10)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

print('test acc:', test_acc)

In [ ]:
img = test_images[1]
img.shape

In [ ]:
img = (np.expand_dims(img, 0))
img.shape

In [ ]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

predictions = probability_model.predict(test_images)

In [ ]:
predictions[0]

In [ ]:
np.argmax(predictions[0])

In [ ]:
test_labels[0]

In [ ]:
class_names[9]

As we can see, the model predicted it correctly
continue this project with these [docs](https://tensorflow.org/tutorials/keras/classification)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

In [ ]:
# Grab an image from the test dataset.
img = test_images[1]

print(img.shape)

In [ ]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))

print(img.shape)

In [ ]:
predictions_single = probability_model.predict(img)

print(predictions_single)

In [ ]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)
plt.show()

In [ ]:
pred = np.argmax(predictions_single[0])
pred

In [ ]:
# Or, with the class names that we have,
class_names[pred]

In [ ]:
# save this model using tfjs converter
# import tensorflowjs as tfjs

In [ ]:
model.save('../models/classify_clothes/classify_clothes.keras')